# WML Federated Learning MNIST Party Example

This notebook is intended to be run by the Federated Learning experiment parties.

### Pre-req

Download data set from https://dataplatform.test.cloud.ibm.com/exchange/public/entry/view/903188bb984a30f38bb889102a1baae5?context=cpdaas
    
Upload mnist-keras-train.pkl and mnist-keras-test.pkl to the same project where you are running this party notebook.

### Check Data Files Exist

In [1]:
!ls /project_data/data_asset

mnist-keras-test.pkl  mnist-keras-train.pkl


### Paste Variables From Aggregator Notebook

**Note:** You need to fill in the credentials for the notebook to run properly.

In [2]:
# hostname of CP4D cluster
CP4D_HOST = "" # <host URL>
CP4D_URL = "https://" + CP4D_HOST

# CP4D user
WS_USER = "" # <username>
WS_PASSWORD = "" # <password>

TRAINING_ID = "" # Get this from the FL model training UI 
RTS_ID = "" # Get this from the FL model training UI

### Install the IBM WML SDK with FL

In [3]:
!pip install ibm_watson_machine_learning

### Install the basic needed libs for the Party to work

In [4]:
!pip install environs parse websockets jsonpickle pandas pytest pyYAML requests pathlib2 psutil setproctitle tabulate lz4 opencv-python gym ray==0.8.0 cloudpickle==1.3.0 image

     |████████████████████████████████| 79 kB 1.4 MB/s eta 0:00:01
     |████████████████████████████████| 465 kB 8.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.8 MB 5.7 MB/s eta 0:00:01
     |████████████████████████████████| 49.5 MB 55.6 MB/s eta 0:00:01                   | 7.0 MB 35.0 MB/s eta 0:00:02                     | 13.0 MB 35.0 MB/s eta 0:00:02               | 15.0 MB 35.0 MB/s eta 0:00:01��███                  | 21.5 MB 35.0 MB/s eta 0:00:01████                 | 23.1 MB 35.0 MB/s eta 0:00:01     |█████████████████▎              | 26.8 MB 19.0 MB/s eta 0:00:02�███████████████▏             | 28.1 MB 19.0 MB/s eta 0:00:02█████▌        | 36.3 MB 19.0 MB/s eta 0:00:01��████████████████▉       | 38.4 MB 19.0 MB/s eta 0:00:01 | 40.7 MB 19.0 MB/s eta 0:00:01███████████▏   | 43.5 MB 19.0 MB/s eta 0:00:01███████████████████████████▏ | 46.6 MB 55.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 15.9 MB/s eta 0:00:01
     |█████████████████████████████

  Created wheel for psutil: filename=psutil-5.7.3-cp37-cp37m-linux_x86_64.whl size=290968 sha256=21c6478612e3746aeca1d114e3e2358aff72157404e5c8347e37f494a503366c
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/f0/4b/57/d037639271e326da3e80e0ddc37bd3d5da7bbf48b3173be557
  Created wheel for setproctitle: filename=setproctitle-1.1.10-cp37-cp37m-linux_x86_64.whl size=33197 sha256=085216a24d727fcf4a7993f75e286b55cb83648df42d137403d3e1824903bbc7
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/02/79/9a/ad2d16e975ac2a43458abc7f87089779308c3018f92bff11b4
  Created wheel for gym: filename=gym-0.17.3-py3-none-any.whl size=1654654 sha256=f9ae7b6d57eaada51540941f77d2588faa4dee1fa817dffd699ce443dd65b582
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/d1/81/4b/dd9c029691022cb957398d1f015e66b75e37637dda61abdf58
  Created wheel for image: filename=image-1.5.33-py2.py3-none-any.whl size=19482 sha256=0566c71582b5b0664f3e74ad64389baf7cb2dadd5f47be10fd78aadfa9663be6
  Stored in directory: /t

### Install the needed frameworks

In [5]:
!pip install tensorflow==2.1.0 scikit-learn==0.23.1 keras==2.2.4 numpy==1.17.4 scipy==1.4.1 

     |████████████████████████████████| 312 kB 1.6 MB/s eta 0:00:01
     |████████████████████████████████| 20.0 MB 11.9 MB/s eta 0:00:01█▉                         | 4.3 MB 11.9 MB/s eta 0:00:02B 11.9 MB/s eta 0:00:01       | 12.6 MB 11.9 MB/s eta 0:00:01��████████████████████   | 18.1 MB 11.9 MB/s eta 0:00:01
     |████████████████████████████████| 26.1 MB 24.6 MB/s eta 0:00:01█▋                         | 5.4 MB 24.6 MB/s eta 0:00:01MB 24.6 MB/s eta 0:00:01�███▊                     | 8.8 MB 24.6 MB/s eta 0:00:01�████████▎                   | 10.0 MB 24.6 MB/s eta 0:00:01     |███████████████                 | 12.3 MB 24.6 MB/s eta 0:00:01�▏           | 16.4 MB 24.6 MB/s eta 0:00:01B 24.6 MB/s eta 0:00:01 | 22.0 MB 24.6 MB/s eta 0:00:01


ERROR: sparkmagic 0.15.0 requires nose, which is not installed.
ERROR: hdijupyterutils 0.12.9 requires jupyter>=1, which is not installed.
ERROR: hdijupyterutils 0.12.9 requires nose, which is not installed.
  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5
  Attempting uninstall: scipy
    Found existing installation: scipy 1.5.0
    Uninstalling scipy-1.5.0:
      Successfully uninstalled scipy-1.5.0


### Import the Party, and ensure it is loaded

In [6]:
import ibmfl.party_env_validator
from ibmfl.party.party import Party

No module named 'scikit-learn' Required version is  0.23.1


Using TensorFlow backend.


No module named 'PyYAML'
No module named 'diffprivlib'
No module named 'opencv-python'


### Define a Data Handler, and write it to the local working directory of this notebook

In [7]:
%%writefile mnist_keras_data_handler.py
from keras.preprocessing.image import ImageDataGenerator
import logging

import numpy as np
from keras.utils import np_utils

from ibmfl.data.data_handler import DataHandler
from ibmfl.util.datasets import load_mnist

logger = logging.getLogger(__name__)



class MnistTFDataHandler(DataHandler):
    """
       Data handler for MNIST dataset.
       """

    def __init__(self, data_config=None, channels_first=False):
        super().__init__()
        self.file_name = None
        if data_config is not None:
            if 'train_file' in data_config:
                self.train_file_name = data_config['train_file']
            if 'test_file' in data_config:
                self.test_file_name = data_config['test_file']

    def get_data(self, nb_points=500):
        """
        Gets pre-process mnist training and testing data. Because this method
        is for testing it takes as input the number of datapoints, nb_points,
        to be included in the training and testing set.

        :param: nb_points: Number of data points to be included in each set
        :type nb_points: `int`
        :return: training data
        :rtype: `tuple`
        """
        if self.file_name is None:
            (x_train, y_train), (x_test, y_test) = load_mnist()
            # Reduce datapoints to make test faster
            x_train = x_train[:nb_points]
            y_train = y_train[:nb_points]
            x_test = x_test[:nb_points]
            y_test = y_test[:nb_points]
        else:
            try:
                logger.info(
                    'Loaded training data from ' + str(self.file_name))
                data_train = np.load(self.file_name)
                with open("MNIST-pkl/mnist-keras-train.pkl", 'rb') as f:
                    (x_train, y_train)= pickle.load(f)

                with open("MNIST-pkl/mnist-keras-train.pkl", 'rb') as f:
                    (x_test, y_test)= pickle.load(f)
                
            except Exception:
                raise IOError('Unable to load training data from path '
                              'provided in config file: ' +
                              self.file_name)

        # Add a channels dimension
        import tensorflow as tf
        x_train = x_train[..., tf.newaxis]
        x_test = x_test[..., tf.newaxis]

        print('x_train shape:', x_train.shape)
        print(x_train.shape[0], 'train samples')
        print(x_test.shape[0], 'test samples')

        return (x_train, y_train), (x_test, y_test)

Writing mnist_keras_data_handler.py


### Verify Data Handler Exists

In [8]:
!ls -al

total 4
drwxr-x---. 2 wsuser  watsonstudio   41 Nov 17 04:11 .
drwxrwx---. 1 wsbuild wsbuild        88 Nov 17 04:11 ..
-rw-r-----. 1 wsuser  watsonstudio 2496 Nov 17 04:11 mnist_keras_data_handler.py


### Create The Party Configuration

In [9]:
from pathlib import Path
working_dir = !pwd
pwd = working_dir[0]

party_config = {
  "aggregator": {
    "ip": CP4D_HOST  + "/ml/v4/trainings/" + TRAINING_ID
  },
  "connection": {
    "info": {
      "id": RTS_ID,
    }
  },
  "data": {
    "info": {
      "train_file": "/project_data/data_asset/mnist-keras-train.pkl",
      "test_file": "/project_data/data_asset/mnist-keras-test.pkl"
    },
    "name": "MnistTFDataHandler",
    "path": pwd + "/mnist_keras_data_handler.py"
  },
  "local_training": {
    "name": "LocalTrainingHandler",
    "path": "ibmfl.party.training.local_training_handler"
  },
  "protocol_handler": {
    "name": "PartyProtocolHandler",
    "path": "ibmfl.party.party_protocol_handler"
  }
}

In [10]:
print(party_config)

{'aggregator': {'ip': 'wslnov4-cpd-wslnov4.apps.wml-fl-ocp-svl-fyre.os.fyre.ibm.com/ml/v4/trainings/bf53d601-b116-4a77-9735-b40350eba49e'}, 'connection': {'info': {'id': '327a5c10-42c0-4136-8342-ca055581ab65'}}, 'data': {'info': {'train_file': '/project_data/data_asset/mnist-keras-train.pkl', 'test_file': '/project_data/data_asset/mnist-keras-test.pkl'}, 'name': 'MnistTFDataHandler', 'path': '/home/wsuser/work/mnist_keras_data_handler.py'}, 'local_training': {'name': 'LocalTrainingHandler', 'path': 'ibmfl.party.training.local_training_handler'}, 'protocol_handler': {'name': 'PartyProtocolHandler', 'path': 'ibmfl.party.party_protocol_handler'}}


### Obtain CP4D Authentication Token

In [11]:
wml_credentials = {
  "username": WS_USER,
  "password": WS_PASSWORD,
  "instance_id" : "wml_local",
  "url": "https://" + CP4D_HOST,
  "version": "3.5"
}

from ibm_watson_machine_learning import APIClient
wml_client = APIClient(wml_credentials)
auth_token = "Bearer " + wml_client.wml_token


2020-11-17 04:12:45,888 - ibmfl.util.config - WARNING - Yaml configuration file not found: log_config.yaml


In [ ]:
print("Token: " + auth_token)

### Establish Connection To Aggregator

In [13]:
p = Party( config_dict = party_config, token = auth_token, self_signed_cert=True  )

2020-11-17 04:12:45,922 | 1.0.0 | INFO | ibmfl.util.config                                  | No model config provided for this setup.
2020-11-17 04:12:45,925 | 1.0.0 | INFO | ibmfl.util.config                                  | No fusion config provided for this setup.
2020-11-17 04:12:45,965 | 1.0.0 | INFO | ibmfl.connection.websockets_connection             | Websockets Sender initialized
2020-11-17 04:12:45,976 | 1.0.0 | INFO | ibmfl.connection.websockets_connection             | WSConnection : Initialize Party Communications
2020-11-17 04:12:45,982 | 1.0.0 | INFO | ibmfl.connection.websockets_connection             | **** PartySendLoopThread
2020-11-17 04:12:45,987 | 1.0.0 | INFO | ibmfl.connection.websockets_connection             | **** PartyRecvLoopThread
2020-11-17 04:12:45,989 | 1.0.0 | INFO | ibmfl.party.party                                  | Party initialization successful
2020-11-17 04:12:46,609 | 1.0.0 | INFO | ibmfl.connection.websockets_connection             | Receiv

After the message "Received Heartbeat from Aggregator" appears, then the Party is ready to start.

### Start Training

In [14]:
p.start()

2020-11-17 04:12:51,117 | 1.0.0 | INFO | ibmfl.party.party                                  | Party not registered yet.
2020-11-17 04:12:51,121 | 1.0.0 | INFO | ibmfl.party.party                                  | Registering party...
2020-11-17 04:12:51,126 | 1.0.0 | INFO | ibmfl.connection.websockets_connection             | Sending serialized message to aggregator
2020-11-17 04:12:54,630 | 1.0.0 | INFO | ibmfl.connection.websockets_connection             | Received serialized message as response: <ibmfl.message.message.Message object at 0x7f858267efd0>
2020-11-17 04:12:54,641 | 1.0.0 | INFO | ibmfl.party.party                                  | Model Package found, placing in working_dir /home/wsuser/work
2020-11-17 04:12:54,825 | 1.0.0 | INFO | ibmfl.party.party                                  | Checking for model config in payload.
2020-11-17 04:12:54,828 | 1.0.0 | INFO | ibmfl.party.party                                  | {'cls_ref': <class 'ibmfl.model.tensorflow_fl_model.Tens

# <hr>
Copyright © 2020 IBM. This notebook and its source code are released under the terms of the MIT License.
 <br><br>
<div style="background:#F5F7FA; height:110px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Love this notebook? </span>
<span style="font-size:15px;color:#152935;float:right;margin-right:40px;">Don't have an account yet?</span><br>
<span style="color:#5A6872;">Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style="border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;"><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
</div>